In [1]:
import os, sys
import json
import numpy as np
import re
from matplotlib import pyplot as plt
from matplotlib import colors
import glob
from collections import defaultdict
import random
%matplotlib inline 

# plot_task

In [2]:
cmap = colors.ListedColormap(
        ['#FFFFFF', '#000000', '#0074D9','#FF4136','#2ECC40','#FFDC00',
         '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])
norm = colors.Normalize(vmin=0, vmax=9)

### Function to initialize an axis ###
######################################
def initAxis(ax, data, title):
    offSet = -.5 # Set offset value (for bug)
    ax.grid() # Show grid
    ax.set_xticklabels([]) # Remove x labels
    ax.set_yticklabels([]) # Remove y labels
    shape = np.asarray(data).shape # Get shape 
    ax.set_xticks(np.arange(offSet, shape[1]))
    ax.set_yticks(np.arange(offSet, shape[0]))
    ax.set_title(title + "\nShape: " + str(shape))
    ax.set_xlabel(shape[1]) # Set x label
    ylbl = ax.set_ylabel(shape[0]) # Set y label
    ylbl.set_rotation(0) # Reset y label rotation
    ax.tick_params(length=0) # Set tick size to zero
    ax.imshow(data+1, cmap=cmap, norm=norm) # Show plot
    
def plot_img(img, title=''):
    fig, axs = plt.subplots(1, 1)
    initAxis(axs, img, title)
    plt.tight_layout(pad=3.0)
    plt.show()
    
### Function to plot a task ###
###############################
def plot_task(task):
    """
    Plots the first train and test pairs of a specified task,
    using same color scheme as the ARC app
    """
    #
    fig, axs = plt.subplots(1, 4, figsize=(15,10))
    # Train input/output
    trnIn = task['train'][0]['input']
    trnOut = task['train'][0]['output']
    initAxis(axs[0], trnIn, 'Train Input')
    initAxis(axs[1], trnOut, 'Train Output')
    #
    # Test input/output
    tstIn = task['test'][0]['input']
    tstOut = task['test'][0]['output']
    initAxis(axs[2], tstIn, 'Test Input')
    initAxis(axs[3], tstOut, 'Test Output')
    #
    plt.tight_layout(pad=3.0)
    plt.show()

In [3]:
def task_id(filename):
    return os.path.splitext(os.path.basename(filename))[0]

In [4]:
# files = glob.glob("../data/training/*.json")
# for task_file in files:
#     with open(task_file, 'r') as f:
#         task = json.load(f)
#         print(f'{task_id(task_file)}:')
#         plot_task(task)
#         plt.close()

# TestKit

In [5]:
def test_task(ID, solve_fn):
    task_filename = f'./data/training/{ID}.json'
    with open(task_filename, 'r') as file:
        task = json.load(file)
    inputs = [task['train'][i]['input'] for i in range(len(task['train']))]
    inputs += [task['test'][i]['input'] for i in range(len(task['test']))]
    for inimg in inputs:
        inimg = np.array(inimg)
        outimg = solve_fn(inimg)
        plot_img(inimg)
        plot_img(outimg)

In [6]:
def get_an_in_img(ID):
    task_filename = f'../data/training/{ID}.json'
    with open(task_filename, 'r') as file:
        task = json.load(file)
    inputs = [task['train'][i]['input'] for i in range(len(task['train']))]
    return np.array(random.choice(inputs))

# OpKit

In [7]:

def img_create(h, w, color=0):
    return np.full((h, w), fill_value=color, dtype=int)


def img_overlay(base, overlay, pos=(0, 0)):
    underlay = base[pos[0]:pos[0] + overlay.shape[0],
                    pos[1]:pos[1] + overlay.shape[1]]
    overlay = np.where(overlay == -1, underlay, overlay)
    base[pos[0]:pos[0] + overlay.shape[0],
         pos[1]:pos[1] + overlay.shape[1]] = overlay

def img_overlay_color(base, overlay_mask, pos=(0, 0), color=None):
    underlay = base[pos[0]:pos[0] + overlay_mask.shape[0],
                    pos[1]:pos[1] + overlay_mask.shape[1]]
    overlay = np.where(overlay_mask == -1, 
                       underlay, 
                       img_create(*overlay_mask.shape, color=color if color else img_major_color(base)))
    base[pos[0]:pos[0] + overlay.shape[0],
         pos[1]:pos[1] + overlay.shape[1]] = overlay


def img_hist(img):
    d = defaultdict(int)
    for i, j in np.ndindex(img.shape):
        d[img[i, j]] += 1
    return dict(d)


def img_major_color(img):
    hist = img_hist(img)
    return max(hist, key=hist.get)


def img_subimg(img, ulc, lrc):
    return img[ulc[0]:lrc[0]+1, ulc[1]:lrc[1]+1]


def img_interest_area(img, bgc=None):
    mc = img_major_color(img) if bgc is None else bgc
    foreground = np.where(img != mc)
    return (min(foreground[0], default=0), min(foreground[1], default=0)), (max(foreground[0], default=0), max(foreground[1], default=0))


def img_colors(img):
    return set(img_hist(img).keys())


def img_filter(img, color, bg_color=None):
    bgc = bg_color if bg_color is not None else img_major_color(img)
    return np.vectorize(lambda c: c if c == color else bgc)(img)


def img_color_density(img, color):
    fimg = img_filter(img, color)
    ulc, lrc = img_interest_area(fimg)
    area = abs(ulc[0] - lrc[0]) * abs(ulc[1] - lrc[1])
    count = img_hist(img).get(color, 0)
    return count / area


def img_unicolor_objs(img):
    objs_colors = img_colors(img)
    objs_colors.remove(img_major_color(img))
    for obj_color in objs_colors:
        fimg = img_filter(img, obj_color)
        yield img_subimg(fimg, *img_interest_area(fimg))


def img_clear_color(img, color):
    return np.vectorize(lambda c: -1 if c == color else c)(img)


        


# Task 67e8384a

In [8]:
def solve_67e8384a(inimg):
    outimg = img_create(inimg.shape[0]*2, inimg.shape[1]*2)
    
    overlay = inimg
    img_overlay(outimg, overlay, pos=(0, 0))
    overlay = np.flipud(overlay)
    img_overlay(outimg, overlay, pos=(inimg.shape[0], 0))
    overlay = np.fliplr(overlay)
    img_overlay(outimg, overlay, pos=(inimg.shape[0], inimg.shape[1]))
    overlay = np.flipud(overlay)
    img_overlay(outimg, overlay, pos=(0, inimg.shape[1]))
    
    return outimg

In [9]:
# test_task('67e8384a', solve_67e8384a)

# Task 2013d3e2

In [10]:
def solve_2013d3e2(inimg):
    obj = img_subimg(inimg, *img_interest_area(inimg))
    return obj[:obj.shape[0]//2, :obj.shape[1]//2]

In [11]:
# test_task('2013d3e2', solve_2013d3e2)

# Task 5ad4f10b

In [12]:
def solve_5ad4f10b(img):
    
    def _divs(n, m):
        return [1] if m==1 else (_divs(n, m-1) + ([m] if n % m == 0 else []))

    def divs(n):
        for x in _divs(n, n-1): yield x

    bgc = img_major_color(img)
    colors = img_colors(img)
    colors.remove(bgc)
    colors = sorted(list(map(lambda c: (c, img_color_density(img, c)), colors)), key=lambda e: e[1], reverse=True)
    obj_color = colors[0][0]
    pigment = colors[1][0]

    fimg = img_filter(img, obj_color)
    obj = img_subimg(fimg, *img_interest_area(fimg))

    tile_sides = list(set(list(divs(obj.shape[0])) + list(divs(obj.shape[1]))))
    new_obj = []
    for t in reversed(tile_sides):
        new_obj = img_create(obj.shape[0]//t, obj.shape[1]//t)
        unicolor = True
        for i in range(0, obj.shape[0], t):
            for j in range(0, obj.shape[1], t):
                h = img_hist(img_subimg(obj, (i, j), (i+t-1, j+t-1)))
                unicolor = len(h) == 1
                if unicolor:
                    uc = list(h.keys())[0]
                    new_obj[(i//t), (j//t)] = pigment if uc == obj_color else bgc
                else: break
            if not unicolor: break
        if unicolor: break

    return new_obj

In [13]:
# test_task('5ad4f10b', solve_5ad4f10b)

# Task c8cbb738

In [14]:
def solve_c8cbb738(img):
    bgc = img_major_color(img)
    objs = img_unicolor_objs(img)
    objs = list(map(lambda obj: img_clear_color(obj, bgc), objs))
    objs_sizes = (obj.shape for obj in objs)
    combined_size = tuple(map(lambda t: max(t), zip(*objs_sizes)))
    outimg = img_create(combined_size[0], combined_size[1], bgc)
    for obj in objs:
        centering_pos = tuple((np.array(outimg.shape) - np.array(obj.shape)) // 2)
        img_overlay(outimg, obj, centering_pos)
    return outimg

In [15]:
# test_task('c8cbb738', solve_c8cbb738)

# Task 681b3aeb

In [16]:
def solve_681b3aeb(img):
    
    def conv_overlay(img1, img2):
        for i in range(-img2.shape[0], img1.shape[0] + 1):
            for j in range(-img2.shape[1], img1.shape[1] + 1):
                index = np.array((i, j))
                f = np.where((index < 0), (img2.shape + index), (img1.shape - index))
                intersection = np.array(list(map(min, zip(img1.shape, img2.shape, f))))
                out_size = np.array(img1.shape) + np.array(img2.shape) - intersection
                outimg = img_create(*out_size, color=-1)
                pos1 = tuple(np.where(index < 0, -index, 0))
                pos2 = tuple(np.where(index < 0, 0, index))
                img_overlay(outimg, img1, pos1)
                img_overlay(outimg, img2, pos2)
                yield outimg

    def does_match(img, objs):
        if(len(img_hist(img).keys()) > 2):
            return False
        else:
            for obj in objs:
                colors = img_hist(obj)
                colors.pop(-1, 0)
                obj_color = list(colors.keys())[0]
                fimg = img_filter(img, color=obj_color, bg_color=-1)
                fobj = img_subimg(fimg, *img_interest_area(fimg, bgc=-1))
                if(not np.array_equal(obj, fobj)):
                    return False

        return True
    
    bgc = img_major_color(img)
    objs = img_unicolor_objs(img)
    objs = list(map(lambda obj: img_clear_color(obj, bgc), objs))
    for im in conv_overlay(*objs):
        if(does_match(im, objs)):
            return im

In [17]:
# test_task('681b3aeb', solve_681b3aeb)

# Task d4a91cb9

In [18]:
def solve_d4a91cb9(img):
    pass

In [19]:
img = get_an_in_img('d4a91cb9')
plot_img(img)
val, val_count = np.unique(img, return_counts=True)
list(zip(val, val_count))

FileNotFoundError: [Errno 2] No such file or directory: '../data/training/d4a91cb9.json'

In [ ]:
# test_task('d4a91cb9', solve_d4a91cb9)

# Task 6d75e8bb

In [ ]:
def solve_6d75e8bb(img):
    outimg = np.copy(img)
    fill_color = 2
    bgc = img_major_color(img)
    bb = img_interest_area(img)
    obj = img_subimg(img, *bb)
    obj = np.where(obj == bgc, fill_color, obj)
    img_overlay(outimg, obj, bb[0])
    
    return outimg

In [ ]:
test_task('6d75e8bb', solve_6d75e8bb)

# Task 6a1e5592

In [ ]:
def solve_6a1e5592(img):
    pass

In [ ]:
# test_task('6a1e5592', solve_6a1e5592)

# Test

In [ ]:
img = get_an_in_img('6a1e5592')
plot_img(img)

img = img_subimg(img, (3, 0), (img.shape[0], img.shape[1]))
plot_img(img)
cleared_img = img_clear_color(img, 0)
plot_img(cleared_img)
plot_img(img)

In [ ]:
def nex_pixel_on_perimiter(img, previous, current, bgc):
    

def img_find_obj(img):
    bgc = img_major_color(img)
    

def img_extract_objs(img):
    pass